In [1]:
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install text2num
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 9.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.4/389.4 kB 7.9 MB/s eta 0:00:00


In [2]:
# -------------------------------------
# LIBRERÍAS Y CONFIGURACIÓN DE MODELOS
# -------------------------------------
import spacy
import json
from spacy.matcher import Matcher
from text_to_num import text2num
import requests
from datetime import datetime

nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)


In [3]:
# ---------------------------------------
# CONFIGURACIÓN DE INFORMACIÓN UTILIZADA
# ---------------------------------------

MESES = [
    "enero", "febrero", "marzo", "abril", "mayo", "junio",
    "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"
]

AEROLINEAS = [
    "iberia", "latam", "avianca", "copa", "american airlines", "delta", "qatar airways", "emirates", "ryanair", "jetblue",
    "aeromexico", "aeroregional", "air europa", "klm", "jetsmart", "united"
]

# --------------------------
# CONFIGURACIÓN DE MATCHERS
# --------------------------

# ---- FECHA ----
matcher.add(
    "FECHA",
    [[
        {"LIKE_NUM": True},
        {"LOWER": "de"},
        {"LOWER": {"IN": MESES}}
    ]]
)

# ---- AEROLÍNEAS ----
for aerolinea in AEROLINEAS:
    pattern = [{"LOWER": palabra} for palabra in aerolinea.split()]
    matcher.add("AEROLINEA", [pattern])


In [4]:
# ---------------------
# FUNCIONES AUXILIARES
# ---------------------
def normalizar_numero(texto):
  """
    Convierte cantidades expresadas en lenguaje natural o en formato numérico
    a un valor entero. Soporta números escritos en español (ej. "veinte y cinco")
    mediante la librería text2num, garantizando escalabilidad y consistencia
    en el dato final.
  """
  texto = texto.lower().strip()
  if texto.isdigit():
      return int(texto)

  try:
      return text2num(texto, "es")
  except:
      return None

def extraer_ciudad(doc, preposicion):
    """
    Extrae el nombre completo de una ciudad, siendo compatible con ciudades
    formadas por varias palabras y con preposiciones internas, como "La Paz",
    "Buenos Aires", "Ciudad de México" o "Río de Janeiro".
    """
    ARTICULOS = {"la", "el", "los", "las"}

    for i, token in enumerate(doc):
        if token.lower_ != preposicion:
            continue

        ciudad_tokens = []
        j = i + 1

        if j < len(doc) and doc[j].lower_ in ARTICULOS:
            ciudad_tokens.append(doc[j].text)
            j += 1

        if j >= len(doc) or doc[j].pos_ != "PROPN":
            continue

        while j < len(doc) and doc[j].pos_ == "PROPN":
            ciudad_tokens.append(doc[j].text)
            j += 1

        while (
            j + 1 < len(doc)
            and doc[j].lower_ == "de"
            and doc[j + 1].pos_ == "PROPN"
        ):
            ciudad_tokens.append(doc[j].text)
            j += 1

            while j < len(doc) and doc[j].pos_ == "PROPN":
                ciudad_tokens.append(doc[j].text)
                j += 1

        if ciudad_tokens:
            return " ".join(ciudad_tokens)

    return None


def extraer_cantidad(doc):
  """
    Identifica y extrae la cantidad de boletos a partir del contexto del mensaje,
    analizando las palabras que preceden a términos como "boletos" o "pasajes".
    Permite interpretar números simples y compuestos en español
    (ej. "cuatro", "veinte y cinco") y los normaliza a valores enteros.
  """
  PALABRAS_CANTIDAD = {"boletos", "boleto", "billetes", "billete", "pasajes", "pasaje"}
  for i, token in enumerate(doc):
      if token.lower_ in PALABRAS_CANTIDAD:
          numero_tokens = []
          j = i - 1

          while j >= 0 and (
              doc[j].like_num or doc[j].lower_ == "y"
          ):
              numero_tokens.insert(0, doc[j].text)
              j -= 1

          numero_texto = " ".join(numero_tokens).strip()

          if numero_texto.isdigit():
              return int(numero_texto)

          try:
              return text2num(numero_texto, lang="es")
          except:
              return None

  return None

def extraer_ruta_sin_de(doc):
    """
    Extrae origen/destino cuando el usuario no utiliza 'de'.
    """
    STOP_ORIGEN = {
        "billete", "billetes",
        "boleto", "boletos",
        "pasaje", "pasajes",
        "comprar", "compra",
        "necesito", "quiero",
        "viaje", "viajar"
    }

    for i, token in enumerate(doc):
        if token.lower_ == "a" and i > 0 and i + 1 < len(doc):
            # --- destino hacia adelante ---
            destino_tokens = []
            k = i + 1
            while k < len(doc) and doc[k].pos_ == "PROPN":
                destino_tokens.append(doc[k].text)
                k += 1

            # --- origen hacia atrás ---
            origen_tokens = []
            j = i - 1
            while j >= 0:
                if doc[j].pos_ == "PROPN" and doc[j].lower_ not in STOP_ORIGEN:
                    origen_tokens.insert(0, doc[j].text)
                    j -= 1
                else:
                    break

            if origen_tokens and destino_tokens:
                return " ".join(origen_tokens), " ".join(destino_tokens)

    return None, None


def extract_information(text):
  """
  Función principal de procesamiento de lenguaje natural.
  Integra el análisis del texto del usuario para extraer información estructurada
  como origen, destino, fecha, cantidad y aerolínea.
  """
  doc = nlp(text)

  origen = extraer_ciudad(doc, "de")
  destino = extraer_ciudad(doc, "a")

  if origen is None or destino is None:
    o2, d2 = extraer_ruta_sin_de(doc)
    if origen is None:
      origen = o2
    if destino is None:
      destino = d2

  cantidad = extraer_cantidad(doc)
  if cantidad is None:
    cantidad = 1

  fecha = aerolinea = None

  for match_id, start, end in matcher(doc):
      label = nlp.vocab.strings[match_id]
      span = doc[start:end]

      if label == "FECHA" and fecha is None:
          fecha = span.text

      elif label == "AEROLINEA" and aerolinea is None:
          aerolinea = span.text.title()

  return {
      "origen": origen,
      "destino": destino,
      "fecha": fecha,
      "cantidad": cantidad,
      "aerolínea": aerolinea
  }

def normalizar_fecha_formato(fecha_texto):
    """
    Implementación de nuevo formato de fecha.
    Devuelve la fecha en formato DD-MM-AAA
    """
    if not fecha_texto: return None
    MESES = {"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06",
             "julio":"07","agosto":"08","septiembre":"09","octubre":"10","noviembre":"11","diciembre":"12"}
    partes = fecha_texto.lower().replace("de", "").split()
    try:
        dia, mes_nombre = partes[0], partes[1]
        mes_num = MESES.get(mes_nombre, "01")
        anio = datetime.now().year
        if int(mes_num) < datetime.now().month: anio += 1
        return f"{int(dia):02d}-{mes_num}-{anio}"
    except:
        return datetime.now().strftime("%d-%m-%Y")



In [5]:
# ------------------
# ASISTENTE VIRTUAL
# ------------------
def assistant_v1():
    print("Hola, bienvenido a BMJ.SA\n¿Cómo te puedo ayudar?")
    user_input = input(">> ")
    print(">> ",user_input)
    info = extract_information(user_input)

    if info["fecha"] is None:
      print(
          f"\nPerfecto, comienzo la búsqueda de tu viaje a {info['destino']} "
          f"desde {info['origen']} con {info['aerolínea']}."
      )
    else:
      print(
          f"\nPerfecto, comienzo la búsqueda de tu viaje a {info['destino']} "
          f"desde {info['origen']} para el {info['fecha']} "
          f"con {info['aerolínea']}."
      )

    print("\nJSON generado:")
    print(json.dumps(info, indent=4, ensure_ascii=False))


assistant_v1()

Hola, bienvenido a BMJ.SA
¿Cómo te puedo ayudar?
>> Hola
>>  Hola

Perfecto, comienzo la búsqueda de tu viaje a None desde None con None.

JSON generado:
{
    "origen": null,
    "destino": null,
    "fecha": null,
    "cantidad": 1,
    "aerolínea": null
}


In [6]:
# ==========================================
#FUNCIÓN DE BÚSQUEDA Y VALIDACIÓN IATA
# ==========================================
def validar_ciudad(ciudad_input):
    """
    Verifica si una ciudad existe usando la API de Travelpayouts.
    Retorna el código IATA si existe o None si no se encuentra.
    """
    if not ciudad_input:
        return None

    url = "https://autocomplete.travelpayouts.com/places2"

    params = {
        "term": ciudad_input,
        "locale": "es",
        "types[]": ["city", "airport"]
    }

    try:
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()

            if data and len(data) > 0:
                resultado = data[0]
                # nombre_real = resultado.get('name')
                # pais = resultado.get('country_name')
                iata = resultado.get('code')
                # print(f"   [OK] '{ciudad_input}' identificada como: {nombre_real}, {pais} ({iata})")
                return iata
            else:
                #print(f"La ciudad '{ciudad_input}' NO existe o no fue encontrada en la base de datos.")
                return None
        else:
            print("Error al conectar con el servicio de validación.")
            return None

    except Exception as e:
        print(f"Error de conexión: {e}")
        return None

In [16]:
# ==========================================
# ASISTENTE MODIFICADO
# ==========================================
def assistant_v2():
    print("Hola, bienvenido a BMJ.SA")
    print("¿Cómo te puedo ayudar?")
    print("Recurda que debes escribir 'salir' para finalizar la conversación.\n")

    while True:
        user_input = input(">> ")

        user_input_normalized = user_input.strip().lower()

        if user_input_normalized == "salir":
            print("Gracias por usar el asistente. ¡Buen viaje!")
            break

        print("Espera un momento, estoy procesando tu solicitud...")
        print("Me has dicho:", user_input, "\nAhora te mostraré los detalles de tu vuelo: ")

        info_raw = extract_information(user_input)

        if info_raw["origen"]is None:
            print("No se ha proporcionado [ciudad de origen] en la solicitud...")
            return

        if info_raw["destino"]is None:
            print("No se ha proporcionado [ciudad de destino] en la solicitud...")
            return

        iata_from = validar_ciudad(info_raw["origen"])
        if not iata_from:
            print("No podemos procesar la cuidad de origen no es válida.")
            return

        iata_to = validar_ciudad(info_raw["destino"])
        if not iata_to:
            print("No podemos procesar la cuidad de destino no es válida.")
            return

        fecha_fmt = normalizar_fecha_formato(info_raw["fecha"])

        json_final = {
            "Origen": info_raw["origen"].title() if info_raw["origen"] else "Desconocido",
            "Ciudad Destino": info_raw["destino"].title() if info_raw["destino"] else "Desconocido",
            "Nombre Ciudad IATA From": iata_from,
            "Nombre Ciudad IATA To": iata_to,
            "Fecha": fecha_fmt,
            "Pax": info_raw["cantidad"],
            "Aerolinea": info_raw["aerolínea"]
        }

        print("\nJSON GENERADO:")
        print(json.dumps(json_final, indent=4, ensure_ascii=False))
        print("\n¿Hay algo más en lo que pueda ayudarle?")

# Ejecutar
assistant_v2()

Hola, bienvenido a BMJ.SA
¿Cómo te puedo ayudar?
Recurda que debes escribir 'salir' para finalizar la conversación.

>> Boleto desde Lima a Medellin
Espera un momento, estoy procesando tu solicitud...
Me has dicho: Boleto desde Lima a Medellin 
Ahora te mostraré los detalles de tu vuelo: 

JSON GENERADO:
{
    "Origen": "Lima",
    "Ciudad Destino": "Medellin",
    "Nombre Ciudad IATA From": "LIM",
    "Nombre Ciudad IATA To": "MDE",
    "Fecha": null,
    "Pax": 1,
    "Aerolinea": null
}

¿Hay algo más en lo que pueda ayudarle?
>> salir
Gracias por usar el asistente. ¡Buen viaje!
